In [1]:
from langchain_groq import ChatGroq

In [5]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
   model_name="llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the moon.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)






















Nike Careers








































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Langua

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

Here is the extracted job postings in JSON format:

```
[
    {
        "role": "Athlète Nike (Vendeur/euse en magasin)",
        "experience": "",
        "skills": "",
        "description": "Job Location: Claye, Seine-et-Marne, France, Retail Stores"
    },
    {
        "role": "APLA, Supply Chain Sustainability, Senior",
        "experience": "",
        "skills": "",
        "description": "Job Location: Ciudad de México, Ciudad de Mexico, Mexico, Corporate Services"
    },
    {
        "role": "ASSISTANT HEAD COACH",
        "experience": "",
        "skills": "",
        "description": "Job Location: Nanjing, Jiangsu, China, Retail Stores"
    },
    {
        "role": "Abteilungsleiter/in (Apparel Coach)",
        "experience": "",
        "skills": "",
        "description": "Job Location: Ingolstadt, Bavaria, Germany, Retail Stores"
    },
    {
        "role": "Abteilungsleiter/in (BOH Coach)",
        "experience": "",
        "skills": "",
        "description": "Job Loca

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Athlète Nike (Vendeur/euse en magasin)',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Claye, Seine-et-Marne, France, Retail Stores'},
 {'role': 'APLA, Supply Chain Sustainability, Senior',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Ciudad de México, Ciudad de Mexico, Mexico, Corporate Services'},
 {'role': 'ASSISTANT HEAD COACH',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Nanjing, Jiangsu, China, Retail Stores'},
 {'role': 'Abteilungsleiter/in (Apparel Coach)',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Ingolstadt, Bavaria, Germany, Retail Stores'},
 {'role': 'Abteilungsleiter/in (BOH Coach)',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Wertheim, Baden-Wurttemberg, Germany, Retail Stores'},
 {'role': 'Abteilungsleiter/in (Coach)',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Landquart, Graubunden, Switzerland, Retail Stores'},
 {'ro

In [13]:
type(json_res)

list

In [15]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [16]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [33]:
# links = collection.query(
#     query_texts=[job['skills']],  # Wrap in a list
#     n_results=2
# ).get('metadatas', [])

# print(links)


In [34]:
# job = json_res[0]  # or job = job[0] if you already assigned job = json_res
# print(job['skills'])  # This should work now


In [36]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
    Remember you are Mohan, BDE at AtliQ. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm

# Iterate over all jobs
for job in json_res:
    print(f"\n🧠 Processing role: {job['role']}")
    
    # Handle list of skills or string
    skills_query = (
        job['skills'] if isinstance(job['skills'], list)
        else [job['skills']]
    )

    # Combine skill list into a single query string
    query_text = " ".join(skills_query)

    # Query ChromaDB
    links = collection.query(
        query_texts=[query_text],
        n_results=2
    ).get('metadatas', [])

    # Flatten list of metadata dicts to just links
    link_list = [item["links"] for item in links[0]] if links else []

    # Generate cold email
    res = chain_email.invoke({
        "job_description": str(job),
        "link_list": link_list
    })

    print(f"\n📩 Email for {job['role']}:\n{res.content}")


🧠 Processing role: Athlète Nike (Vendeur/euse en magasin)

📩 Email for Athlète Nike (Vendeur/euse en magasin):
Subject: Streamline Your Retail Operations with AtliQ's Expertise

Dear Hiring Manager,

I came across the job posting for an Athlète Nike (Vendeur/euse en magasin) at your Claye, Seine-et-Marne, France location, and I believe AtliQ can help you optimize your retail operations to better support your sales team.

As a business development executive at AtliQ, I'd like to introduce you to our AI-powered solutions that can help you streamline your retail processes, reduce costs, and enhance overall efficiency. Our expertise in automation can help you:

* Improve inventory management and supply chain optimization
* Enhance customer experience through personalized interactions
* Increase sales through data-driven insights and targeted marketing
* Optimize store operations and reduce manual errors

Our portfolio showcases our capabilities in developing innovative solutions for vario